In [1]:
# import libraries

from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import requests
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
import matplotlib
import nltk
import random

In [2]:
def headers():
    i = random.randint(3,5)
    j=random.randint(40,53)
    x=random.randint(2,13)
    url_base = 'https://www.indeed.ca/jobs?q='
    headers ={
            'User-Agent':'Mozilla/'+str(i)+'.0 (Macintosh; Intel Mac OS X 10.'+str(x)+'; rv:53.0) Gecko/20100101 Firefox/'+str(j)+'.0'
                }
    return headers


In [65]:
def text_cleaner(website):
    try:
        html = urllib.request.urlopen(urllib.request.Request(website, headers={'User-Agent': 'Mozilla/5.0'})).read()
        soup = BeautifulSoup(html, "html.parser")
        for script in soup(["script", "style"]):
            script.extract()
        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split())
        text = ' '.join(chunks)
        bytes(text, "utf-8").decode("unicode_escape")
        text = bytes(text, "utf-8").decode("unicode_escape")
        text = re.sub("[-:.!,*+%?//#·$;|]"," ", text)
    except:
        text = 'Unknown'
    return text

In [66]:
def skills_info_test(city = None, state = None, job_title = None):
    final_job = str(job_title)
    columns = ["job_title","job_link","job_description","company_name","location","summary","salary"]
    df = pd.DataFrame(columns = columns)
    num = (len-(df)+1)
    
     # Now find out how many jobs there were
    
    
    final_job = str(job_title)
    
    # Make sure the city specified works properly if it has more than one word (such as San Francisco)
    if city is not None:
        final_city = city.split() 
        final_city = '+'.join(word for word in final_city)
        final_site_list = ['http://ca.indeed.com/jobs?q=', final_job, '&l=', final_city,
                    '%2C+', state] # Join all of our strings together so that indeed will search correctly
    else:
        final_site_list = ['http://ca.indeed.com/jobs?q="', final_job, '"']

    final_site = ''.join(final_site_list) # Merge the html address together into one string
    print(final_site)

    base_url = 'http://ca.indeed.com'
    
  
    
    for start in range(0,200,10):
       
    
    # Make sure the city specified works properly if it has more than one word (such as San Francisco)
        if city is not None:
            final_city = city.split() 
            final_city = '+'.join(word for word in final_city)
            final_site_list = ['http://ca.indeed.com/jobs?q=', final_job, '&l=', final_city,
                    '%2C+', state,'&start=',str(start)] # Join all of our strings together so that indeed will search correctly
        else:
            final_site_list = ['http://ca.indeed.com/jobs?q="', final_job, '"','&start=', str(start)]

        final_site = ''.join(final_site_list) # Merge the html address together into one string
        print(final_site)

        base_url = 'http://ca.indeed.com'
    
        try:
            html = requests.get(final_site, headers = headers()) # Open up the front page of our search first
        except:
            print('That city/state combination did not have any jobs. Exiting . . .') # In case the city is invalid
            return
        
        soup = BeautifulSoup(html.text, 'html.parser') # Get the html from the first page
    
        job_link_area = soup.find(id = 'resultsCol')
        
        #target = BeautifulSoup(html.text, "html.parser") # Get the html from the first page
        #targetElements = target.findAll('div',attrs={'class':'row result'})
        
        
        
        #for elem in targetElements:
            #job_link = "%s%s" % (base_url,elem.find('a').get('href'))
            #job_post.append(job_link)
            
        for div in soup.find_all(name = "div", attrs = {"class":"row"}):
            #specifying row num for index of job posting in dataframe
            job_post = []
            num = (len(df)+1)
            
            #creating an empty list to hold the data for each posting
            
            
        
            # grabbing job title
        
            for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
                job_post.append(a["title"])
        
            # grabbing job link
            job_descriptions = [] 
            for a in div.find_all(name = "a", attrs = {"data-tn-element":"jobTitle"}):
                job_post.append(str(base_url) + str(a["href"]))
                
                final_description = text_cleaner(str(base_url) + str(a['href']))
                job_post.append(final_description)
                #sleep(0)
             # grabbing detailed description
            
            
            
            #grabbing company name
        
            company = div.find_all(name = "span", attrs = {"class":"company"})
        
            if len(company) > 0:
                for b in company:
                    job_post.append(b.text.strip())
            else:
                sec_try = div.find_all(name = "span", attrs = {"class":"result-link-source"})
            
                for span in sec_try:
                    job_post.append(span.text)
                
            # grabbing location name
        
            c = div.findAll('span',attrs = {'class':'location'})
        
            for span in c:
            
                job_post.append(span.text)
        
            #grabing summary text
        
            d = div.findAll('span', attrs = {'class':'summary'})
            for span in d:
                job_post.append(span.text.strip())
                
           
   
            # grabbing salary
    
            try:
                job_post.append(div.find('nobr').text)
        
            except:
                try:
                    div_2 = div.find(name = 'div', attrs = {"class":"sjcl"})
                    div_3 = div_2.find("div")
                    job_post.append(div_3.text.strip())
                except:
                    job_post.append("Nothing_Found")
                
            #appending list of job post to a data frame at index num
        
            df.loc[num] = job_post
        
    
    
    return(df)

In [67]:
pd.set_option('display.max_colwidth', -1)
df1 = skills_info_test(city = 'Toronto', state = 'ON', job_title = "Data+Scientist")


http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=0
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=10
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=20
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=30
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=40
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=50
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=60
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=70


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=80
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=90
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=100
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=110
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=120
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=130
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=140
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=150
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=160
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=170
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=180
http://ca.indeed.com/jobs?q=Data+Scientist&l=Toronto%2C+ON&start=190


In [68]:
df1

job_title  \
1    Data Scientist                                                    
2    Real Estate Agent / Realtor / 100% Commission                     
3    Data Scientist                                                    
4    Data Scientist                                                    
5    Data Scientist: Advanced Analytics                                
6    Data Scientist                                                    
7    Machine Learning Developer / Data Scientist                       
8    Data Scientist                                                    
9    Data Scientist                                                    
10   Research Scientist, Google Brain (Canada)                         
11   Data Scientist                                                    
12   Data Scientist                                                    
13   Data Scientist                                                    
14   Software Development Manager                                      
15   Chief Data Scientist (Director) - 19283                           
16   Chief Data Scientist (Director) - 19283                           
17   Real Estate Agent / Realtor / 100% Commission                     
18   Data Scientist                                                    
19   Data Scientist                                                    
20   Data Scientist – Customer Insights                                
21   Data Scientist                                                    
22   People Data Scientist                                             
23   Data Scientist (Toronto)                                          
24   Data Scientist                                                    
25   Business Intelligence QA Analyst - PriceMetrix                    
26   Data Engineer                                                     
27   Machine Learning Engineer / Researcher                            
28   Data Scientist                                                    
29   Software Development Manager                                      
30   Data Scientist                                                    
..              ...                                                    
271  Software Development Manager                                      
272  Data Scientist                                                    
273  Sports Analytics Co-op                                            
274  Data Developer                                                    
275  Financial Reporting Analyst                                       
276  Data Science Manager                                              
277  Summer Student - Fiona Moola                                      
278  Digital Marketing Specialist                                      
279  Vice President of Engineering                                     
280  Intermediate Environmental Technologist                           
281  QA Engineer                                                       
282  Summer Student- Dr. Shannon Scratch                               
283  Real Estate Agent / Realtor / 100% Commission                     
284  Chief Data Scientist (Director) - 19283                           
285  Data Scientist                                                    
286  Chief Data Scientist (Director) - 19283                           
287  Real Estate Agent / Realtor / 100% Commission                     
288  Post-Doctoral Fellow, Electrical and Computer Engineering         
289  R & D Scientist - Chemistry                                       
290  Lead Data Scientist - Growing Software Firm                       
291  BI Developer                                                      
292  Mgr, Store Layout Analytics & Strategy                            
293  INORGANICS SENIOR LABORATORY MANAGER                              
294  Senior Big Data Technology Engineer - Data & Analytics - 245698   
295  Int. Python (Django) a

In [16]:
df1.to_csv('job_data_science.csv')

In [17]:
df2 = skills_info_test(city = 'Toronto', state = 'ON', job_title = "Data+Engineer")

http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=0
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=10
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=20
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=30
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=40
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=50
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=60
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=70
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=80
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=90
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=100
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=110
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=120
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=130
h

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=160
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=170
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=180
http://ca.indeed.com/jobs?q=Data+Engineer&l=Toronto%2C+ON&start=190


In [18]:
df2.to_csv('job_data_engineer.csv',encoding = 'utf-8')

In [19]:
df3 = skills_info_test(city = 'Toronto', state = 'ON', job_title = "Accounting")

http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=0
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=10
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=20
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=30
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=40
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=50
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=60
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=70
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=80
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=90
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=100
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=110
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=120


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: invalid escape sequence '\ '
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: invalid escape sequence '\ '
  if sys.path[0] == '':


http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=130
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=140
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=150
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=160
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=170
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=180
http://ca.indeed.com/jobs?q=Accounting&l=Toronto%2C+ON&start=190


In [20]:
df3.to_csv('job_accounting.csv',encoding = 'utf-8')

In [46]:
import nltk 

In [47]:
import string


In [63]:
df1["tokenized1"] = (line.strip() for line in df1["job_description"].splitlines())

AttributeError: 'Series' object has no attribute 'splitlines'

In [70]:
from sklearn.feature_extraction.text import CountVectorizer


In [73]:
bow_transformer = CountVectorizer().fit(df1["job_description"])

In [75]:
print (len(bow_transformer.vocabulary_))

5212


In [89]:
bow1 = bow_transformer.transform(df1["job_description"])

In [90]:
print (bow1)
print (bow1.shape)

  (0, 14)	1
  (0, 23)	1
  (0, 59)	1
  (0, 68)	1
  (0, 176)	1
  (0, 240)	1
  (0, 273)	1
  (0, 314)	6
  (0, 325)	1
  (0, 333)	1
  (0, 338)	1
  (0, 357)	1
  (0, 389)	1
  (0, 400)	4
  (0, 406)	1
  (0, 425)	1
  (0, 434)	1
  (0, 442)	3
  (0, 443)	1
  (0, 573)	1
  (0, 588)	1
  (0, 596)	1
  (0, 611)	1
  (0, 618)	1
  (0, 628)	1
  :	:
  (299, 4343)	1
  (299, 4444)	1
  (299, 4485)	1
  (299, 4488)	1
  (299, 4617)	1
  (299, 4709)	1
  (299, 4726)	4
  (299, 4728)	1
  (299, 4778)	1
  (299, 4783)	1
  (299, 4785)	2
  (299, 4808)	8
  (299, 4898)	1
  (299, 4947)	1
  (299, 4990)	1
  (299, 5008)	1
  (299, 5083)	1
  (299, 5109)	1
  (299, 5112)	1
  (299, 5136)	4
  (299, 5145)	1
  (299, 5146)	1
  (299, 5152)	1
  (299, 5198)	1
  (299, 5201)	1
(300, 5212)


In [95]:
import csv
df_res = pd.read_csv("resume.csv")

In [102]:
df_res

Unnamed: 0  \
0   {0: ['Finn.AI', 'less than 1 year', "Finn.ai is a white-label virtual banking assistant, powered by artificial intelligence. Finn.ai puts a ‘personal banker’ in every customer's pocket, helping them to manage their money wherever they are, whatever they need via a simple, natural conversation."], 1: ['BC Public Service', 1, "As a Machine Learning Engineer:\xa0• Used Supervised Learning algorithms, such as Neural Networks (NN), Random Forest, and\xa0Logistic Regression to classify user comments into specific topics.\xa0• Developed smart A.I. bots with Microsoft Bot Framework to interact with and ingest user\xa0comments from a variety of sources, such as Skype, Slack, and other messaging platforms\xa0• Developed Sentiment Analysis Models in Python and through Microsoft Cognitive Services to\xa0detect user's sentiment on text comments\xa0• Used LDA (Latent Dirchlet Allocation) to discover new topics from user comments\xa0• Utilized Microsoft Azure Machine Learning Studio and SQL server to create an end-to-end\xa0pipeline, starting from a comment ingested from the A.I Bot to classification/clustering and\xa0storage in the SQL server to live report generation in Power B.I. in under 2 seconds. As a Consultant:\xa0• Communicated results by creating interactive reports in R (Shiny apps) and through Microsoft Power B.I.\xa0• Presented product to B.C government executives and various partner companies • Proposed Machine Learning solutions to existing projects"]}                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

In [266]:

import nltk
import math
from sklearn.feature_extraction.text import TfidfVectorizer

import ast
from sklearn.feature_extraction.text import CountVectorizer

In [242]:
mess = df_res['Unnamed: 0'][0]

In [243]:
mess

'{0: [\'Finn.AI\', \'less than 1 year\', "Finn.ai is a white-label virtual banking assistant, powered by artificial intelligence. Finn.ai puts a ‘personal banker’ in every customer\'s pocket, helping them to manage their money wherever they are, whatever they need via a simple, natural conversation."], 1: [\'BC Public Service\', 1, "As a Machine Learning Engineer:\\xa0• Used Supervised Learning algorithms, such as Neural Networks (NN), Random Forest, and\\xa0Logistic Regression to classify user comments into specific topics.\\xa0• Developed smart A.I. bots with Microsoft Bot Framework to interact with and ingest user\\xa0comments from a variety of sources, such as Skype, Slack, and other messaging platforms\\xa0• Developed Sentiment Analysis Models in Python and through Microsoft Cognitive Services to\\xa0detect user\'s sentiment on text comments\\xa0• Used LDA (Latent Dirchlet Allocation) to discover new topics from user comments\\xa0• Utilized Microsoft Azure Machine Learning Studio 

In [244]:
tknd = nltk.word_tokenize(mess)

In [245]:
len(df1["job_description"])

300

In [267]:
document = [mess]
for i in range(1,20):
    
    document.append(df1["job_description"][i])

In [268]:
document

['{0: [\'Finn.AI\', \'less than 1 year\', "Finn.ai is a white-label virtual banking assistant, powered by artificial intelligence. Finn.ai puts a ‘personal banker’ in every customer\'s pocket, helping them to manage their money wherever they are, whatever they need via a simple, natural conversation."], 1: [\'BC Public Service\', 1, "As a Machine Learning Engineer:\\xa0• Used Supervised Learning algorithms, such as Neural Networks (NN), Random Forest, and\\xa0Logistic Regression to classify user comments into specific topics.\\xa0• Developed smart A.I. bots with Microsoft Bot Framework to interact with and ingest user\\xa0comments from a variety of sources, such as Skype, Slack, and other messaging platforms\\xa0• Developed Sentiment Analysis Models in Python and through Microsoft Cognitive Services to\\xa0detect user\'s sentiment on text comments\\xa0• Used LDA (Latent Dirchlet Allocation) to discover new topics from user comments\\xa0• Utilized Microsoft Azure Machine Learning Studio

In [276]:
def tokenizer(text):
    text = re.sub("[-:.!,*+%?//#·$;|/(/)â]"," ", text)
    vectorizer = CountVectorizer(ngram_range = (1, 1),stop_words = 'english')
    analyzer = vectorizer.build_analyzer()
    tokens = analyzer(text)
    return tokens

from sklearn.feature_extraction.text import TfidfVectorizer

sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenizer)

In [277]:
sklearn_representation = sklearn_tfidf.fit_transform(document)


In [278]:
sklearn_representation

<20x1413 sparse matrix of type '<class 'numpy.float64'>'
	with 2949 stored elements in Compressed Sparse Row format>

In [279]:
def cosine_similarity(vector1, vector2):
    dot_product = sum(p*q for p,q in zip(vector1, vector2))
    magnitude = math.sqrt(sum([val**2 for val in vector1])) * math.sqrt(sum([val**2 for val in vector2]))
    if not magnitude:
        return 0
    return dot_product/magnitude
    
    

In [280]:
skl_tfidf_comparisons = []

In [281]:
for count_0, doc_0 in enumerate(sklearn_representation.toarray()):
    for count_1, doc_1 in enumerate(sklearn_representation.toarray()):
        skl_tfidf_comparisons.append((cosine_similarity(doc_0, doc_1), count_0, count_1))

In [282]:
for i in skl_tfidf_comparisons:
    if i[1] != i[2] and i[1] == 0:
        print (i)

(0.080815864237423762, 0, 1)
(0.015692605693793691, 0, 2)
(0.0, 0, 3)
(0.078357781227395062, 0, 4)
(0.0, 0, 5)
(0.033077515815798646, 0, 6)
(0.0, 0, 7)
(0.0, 0, 8)
(0.040365463463557287, 0, 9)
(0.0, 0, 10)
(0.0, 0, 11)
(0.059136548986773708, 0, 12)
(0.0, 0, 13)
(0.043969399383884784, 0, 14)
(0.060929385302460941, 0, 15)
(0.060929385302460941, 0, 16)
(0.015692605693793691, 0, 17)
(0.032321117236944509, 0, 18)
(0.0, 0, 19)
